In [1]:
import torch
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt

In [2]:
data = torch.load("latent_50915880.pt")
x = data["latent"].numpy()[data["splits"]["val_idx"]]
y = data["condition"].numpy()[data["splits"]["val_idx"]]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

FileNotFoundError: [Errno 2] No such file or directory: 'latent_50915880.pt'

In [ ]:
clf = LogisticRegression(class_weight="balanced", n_jobs=-1)
clf.fit(x_train, y_train)

In [ ]:
print("train acc", clf.score(x_train, y_train))
print("test acc", clf.score(x_test, y_test))

In [ ]:
plt.figure(figsize=(7, 7))
plt.imshow(confusion_matrix(clf.predict(x_test), y_test, normalize="true"))
plt.xticks(range(len(np.unique(y_test))), data["condition2name"], rotation=90)
plt.yticks(range(len(np.unique(y_test))), data["condition2name"])
plt.show()

In [ ]:
idxs = np.random.permutation(len(x))[:20000]
tsne = TSNE(n_components=2, n_jobs=-1, init="pca", learning_rate="auto")
tsne.fit(x[idxs])

In [ ]:
plt.figure(figsize=(15, 7))
for cls in np.unique(y[idxs]):
    plt.scatter(*tsne.embedding_[y[idxs]==cls].T, alpha=0.5, label=data["condition2name"][cls])
plt.legend()
plt.show()

In [ ]:
pca = PCA(n_components=2)
pca.fit(x)

In [ ]:
plt.figure(figsize=(15, 7))
for cls in np.unique(y[idxs]):
    plt.scatter(*pca.transform(x[y==cls]).T, label=data["condition2name"][cls])
plt.legend()
plt.show()